In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [2]:
pro = pd.read_csv('properties.csv')
train = pd.read_csv('train.csv')
cpi = pd.read_csv('cpi.csv')
interest= pd.read_csv('interest.csv')
rentIndex = pd.read_csv('rentIndex.csv')
geo = pd.read_csv('geo_attributes.csv')
vacant = pd.read_csv('vacant.csv')
test = pd.read_csv('test.csv')

In [3]:
interest_and_cpi = pd.merge(left=interest, right=cpi, on='Data Series')
interest_and_cpi.head()

,Data Series,InterestRate,CPI
0,2022 Dec,1.48,111.186
1,2022 Nov,1.47,110.959
2,2022 Oct,1.17,109.893
3,2022 Sep,1.03,110.339
4,2022 Aug,0.95,109.863


In [4]:
rentindex_and_vacant = pd.merge(left=rentIndex, right=vacant, on='Data Series')

In [7]:
train_and_properties = pd.merge(left=train, right=pro, on='property_key', how='inner')

In [8]:
train_properties_geo = pd.merge(left=train_and_properties, right=geo, on=['street', 'project', 'district'], how='inner')
train_properties_geo['contractDate'] = pd.to_datetime(train_properties_geo['contractDate'])
train_properties_geo = train_properties_geo.sort_values(by='contractDate', ignore_index=True, ascending=True)
train_properties_geo.head()
len(train_properties_geo)

62948

In [9]:
train_properties_geo.isna().sum()

property_key             0
contractDate             0
price                    0
area                     0
floorRange               0
propertyType             0
district                 0
typeOfArea               0
tenure                   0
street                   0
project                  0
marketSegment            0
lat                      0
lng                      0
num_schools_1km          0
num_supermarkets_500m    0
num_mrt_stations_500m    0
dtype: int64

In [10]:
datetime, years, months = convert_dataseries_datetime(interest_and_cpi)
interest_and_cpi['contractDate'] = datetime
interest_and_cpi['year'] = years
interest_and_cpi['month']= months
interest_and_cpi.head()

,Data Series,InterestRate,CPI,contractDate,year,month
0,2022 Dec,1.48,111.186,2022-12-01,2022,12
1,2022 Nov,1.47,110.959,2022-11-01,2022,11
2,2022 Oct,1.17,109.893,2022-10-01,2022,10
3,2022 Sep,1.03,110.339,2022-09-01,2022,9
4,2022 Aug,0.95,109.863,2022-08-01,2022,8


# Train + Properties + Geo + Int + CPI

In [11]:
train_properties_geo_intcpi = pd.merge(left=train_properties_geo, right=interest_and_cpi, on='contractDate', how='left')
len(train_properties_geo_intcpi)

62948

In [12]:
train_properties_geo_intcpi.isna().sum()

property_key             0
contractDate             0
price                    0
area                     0
floorRange               0
propertyType             0
district                 0
typeOfArea               0
tenure                   0
street                   0
project                  0
marketSegment            0
lat                      0
lng                      0
num_schools_1km          0
num_supermarkets_500m    0
num_mrt_stations_500m    0
Data Series              0
InterestRate             0
CPI                      0
year                     0
month                    0
dtype: int64

## Data Binning

using data binning on area column in train_properties_geo_intcpi

area: 
     ->  less than 50 sq meter is considered as small
     
     ->  from 50 to 100 sq meter is considered as medium
     
     ->  from 100 to 150 sq meter is considered as medium_large
     
     ->  over 150 is considered as large area
        



In [13]:
cut_lables = ['small', 'medium', 'medium_large', 'large',]
cut_bins = [0, 200 , 400, 600, 1000]
train_properties_geo_intcpi['area binning'] = pd.cut(train_properties_geo_intcpi['area'], bins=cut_bins , labels=cut_lables)


In [14]:
train_properties_geo_intcpi['area'].describe()

count    62948.000000
mean       129.401817
std         86.520823
min         24.000000
25%         83.000000
50%        110.000000
75%        144.000000
max        996.800000
Name: area, dtype: float64

In [16]:
train_properties_geo_intcpi.isna().sum()

property_key             0
contractDate             0
price                    0
area                     0
floorRange               0
propertyType             0
district                 0
typeOfArea               0
tenure                   0
street                   0
project                  0
marketSegment            0
num_schools_1km          0
num_supermarkets_500m    0
num_mrt_stations_500m    0
InterestRate             0
CPI                      0
year                     0
month                    0
area binning             0
dtype: int64

In [17]:
df = train_properties_geo_intcpi.copy()

transaction price of each property key 
From the raw data, if there were multiple contract prices for the same property key, the median value was calculated and used. However, if there was a unique contract for a given date, the actual contract price was utilised.


In [18]:
df_grouped = train_properties_geo_intcpi.groupby(['property_key'])['price'].median().reset_index()

In [19]:
df_grouped.head()

,property_key,price
0,p-000132de8,2800000.0
1,p-00037b28a,1144000.0
2,p-0003815a3,1900000.0
3,p-000542650,2838000.0
4,p-0006b8199,1417500.0


In [20]:
cols = ['property_key',
        'contractDate',
        'year',
        'month',
        'floorRange' ,
        'typeOfArea',
        'area binning',
        'area',
        'propertyType', 
        'district',
       'marketSegment',
       'num_schools_1km', 
        'num_supermarkets_500m',
       'num_mrt_stations_500m',
        'tenure'
       ]
df_features = df[cols].copy()
df_features = df_features.drop_duplicates(subset='property_key', keep='first', ignore_index=True)
df_features.head()

,property_key,contractDate,year,month,floorRange,typeOfArea,area binning,area,propertyType,district,marketSegment,num_schools_1km,num_supermarkets_500m,num_mrt_stations_500m,tenure
0,p-c1ce31019,2018-03-01,2018,3,36-40,Strata,small,102.0,Apartment,1,CCR,0,3,13,99 yrs lease commencing from 2005
1,p-efaeb4d7c,2018-03-01,2018,3,01-05,Strata,small,153.0,Apartment,14,RCR,5,6,4,99 yrs lease commencing from 1997
2,p-896d8c357,2018-03-01,2018,3,01-05,Strata,small,153.0,Condominium,19,OCR,3,1,0,99 yrs lease commencing from 2010
3,p-2d89ee033,2018-03-01,2018,3,06-10,Strata,small,70.0,Condominium,15,OCR,2,0,1,Freehold
4,p-86f48e239,2018-03-01,2018,3,11-15,Strata,small,128.0,Condominium,15,OCR,2,0,1,Freehold


In [21]:
df_merged = pd.merge(left=df_grouped, right=df_features, on='property_key')

In [22]:
df_merged.columns

Index(['property_key', 'price', 'contractDate', 'year', 'month', 'floorRange',
       'typeOfArea', 'area binning', 'area', 'propertyType', 'district',
       'marketSegment', 'num_schools_1km', 'num_supermarkets_500m',
       'num_mrt_stations_500m', 'tenure'],
      dtype='object')

# feature engineering data for ml model

## One hot encoding

In [23]:
onehot = pd.get_dummies(df_merged, columns=['marketSegment', 'typeOfArea', 'area binning'])

## lable encoding

In [24]:
tr = pd.merge(left=train, right=pro, on='property_key', how='outer')

In [25]:
 df['propertyType'].unique()

array(['Apartment', 'Condominium', 'Executive Condominium', 'Terrace',
       'Semi-detached', 'Strata Terrace', 'Detached',
       'Strata Semi-detached', 'Strata Detached'], dtype=object)

In [26]:
lablePropertyType = LabelEncoder().fit(tr['propertyType'])
lableFloorRange = LabelEncoder().fit(tr['floorRange'])
lableTenure = LabelEncoder().fit(tr['tenure'])
import numpy
onehot['propertyType'] = lablePropertyType.transform(onehot['propertyType'])
onehot['floorRange'] = lableFloorRange.transform(onehot['floorRange'])
onehot['tenure'] = lableTenure.transform(onehot['tenure'])



In [27]:
onehot = onehot.drop(['property_key', 'contractDate'], axis=1)


#### This is the final result of one hot and lable encoder

In [28]:
onehot.head()

,price,year,month,floorRange,area,propertyType,district,num_schools_1km,num_supermarkets_500m,num_mrt_stations_500m,tenure,marketSegment_CCR,marketSegment_OCR,marketSegment_RCR,typeOfArea_Land,typeOfArea_Strata,area binning_small,area binning_medium,area binning_medium_large,area binning_large
0,2800000.0,2018,7,1,129.0,0,13,4,4,2,121,0,0,1,0,1,1,0,0,0
1,1144000.0,2018,7,2,120.0,1,17,2,1,0,121,0,1,0,0,1,1,0,0,0
2,1900000.0,2018,4,3,259.0,0,19,9,3,2,52,0,1,0,0,1,0,1,0,0
3,2838000.0,2018,8,1,153.0,0,11,10,5,3,121,1,0,0,0,1,1,0,0,0
4,1417500.0,2021,10,1,110.0,1,23,3,0,0,63,0,1,0,0,1,1,0,0,0


In [29]:
df_train = onehot.copy()

In [46]:
df = df_train.drop(columns=['area'])


# ML

In [87]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.tree import  DecisionTreeRegressor


In [88]:
y = df['price']

In [89]:
X = df_train.drop(['price'], axis=1)

In [90]:
X.head()

,year,month,floorRange,area,propertyType,district,num_schools_1km,num_supermarkets_500m,num_mrt_stations_500m,tenure,marketSegment_CCR,marketSegment_OCR,marketSegment_RCR,typeOfArea_Land,typeOfArea_Strata,area binning_small,area binning_medium,area binning_medium_large,area binning_large
0,2018,7,1,129.0,0,13,4,4,2,121,0,0,1,0,1,1,0,0,0
1,2018,7,2,120.0,1,17,2,1,0,121,0,1,0,0,1,1,0,0,0
2,2018,4,3,259.0,0,19,9,3,2,52,0,1,0,0,1,0,1,0,0
3,2018,8,1,153.0,0,11,10,5,3,121,1,0,0,0,1,1,0,0,0
4,2021,10,1,110.0,1,23,3,0,0,63,0,1,0,0,1,1,0,0,0


In [50]:
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2)

In [83]:
rf = RandomForestRegressor(n_jobs=-1, random_state=41)
rf.fit(train_x, train_y)
pre = rf.predict(test_x)
error = r2_score(test_y, pre)
print(f'r2 score -> {error}')

r2 score -> 0.9189559536335113


In [52]:
len(test)

2331

In [53]:
f_cols = ['property_key',
        'floorRange',
        'typeOfArea',
        'area',
        'propertyType', 
        'district',
       'marketSegment',
       'num_schools_1km', 
        'num_supermarkets_500m',
       'num_mrt_stations_500m',
        'tenure'
        
       ]
modified = df_merged[f_cols]

In [54]:
result = pd.merge(left=test, right=modified, on='property_key', how='left')
result.isna().sum()

property_key               0
contractDate               0
floorRange               657
typeOfArea               657
area                     657
propertyType             657
district                 657
marketSegment            657
num_schools_1km          657
num_supermarkets_500m    657
num_mrt_stations_500m    657
tenure                   657
dtype: int64

# without null value

In [55]:
df_test = result.copy()
df_test = df_test.dropna()
df_test.isna().sum()

property_key             0
contractDate             0
floorRange               0
typeOfArea               0
area                     0
propertyType             0
district                 0
marketSegment            0
num_schools_1km          0
num_supermarkets_500m    0
num_mrt_stations_500m    0
tenure                   0
dtype: int64

# Working with null value

In [56]:
dfnull = pd.merge(left=train, right=pro, on='property_key', how='outer')
len(dfnull)

63596

In [57]:
dfnull = dfnull.drop(['contractDate', 'price'], axis=1)

In [58]:
#  floorRange

nullres = result[result['floorRange'].isna()][['property_key', 'contractDate']]
dfnullmerge = pd.merge(left=nullres, right=dfnull, on='property_key')
dfnullmerge.isna().sum()

property_key     0
contractDate     0
area             0
floorRange       0
propertyType     0
district         0
typeOfArea       0
tenure           0
street           0
project          0
marketSegment    0
dtype: int64

In [59]:
dfnullmerge.columns

Index(['property_key', 'contractDate', 'area', 'floorRange', 'propertyType',
       'district', 'typeOfArea', 'tenure', 'street', 'project',
       'marketSegment'],
      dtype='object')

In [60]:
dfnullmerge_geo =  pd.merge(left=dfnullmerge, right=geo, on=['street', 'project', 'district'], how='left')

In [61]:
f_col = [
    'contractDate'
    ,'property_key',
        'floorRange' ,
        'typeOfArea',
        'area',
        'propertyType', 
        'district',
       'marketSegment',
       'num_schools_1km', 
        'num_supermarkets_500m',
       'num_mrt_stations_500m',
        'tenure'
        ]
dfnullmerge_geo = dfnullmerge_geo[f_col]

In [62]:
dfnullmerge_geo.isna().sum()

contractDate             0
property_key             0
floorRange               0
typeOfArea               0
area                     0
propertyType             0
district                 0
marketSegment            0
num_schools_1km          1
num_supermarkets_500m    1
num_mrt_stations_500m    1
tenure                   0
dtype: int64

In [63]:
dfnullmerge_geo['num_schools_1km'] = dfnullmerge_geo.groupby('marketSegment')['num_schools_1km'].apply(lambda x : x.fillna(x.mode().iloc[0]))
dfnullmerge_geo['num_supermarkets_500m'] = dfnullmerge_geo.groupby('marketSegment')['num_supermarkets_500m'].apply(lambda x : x.fillna(x.mode().iloc[0]))
dfnullmerge_geo['num_mrt_stations_500m'] = dfnullmerge_geo.groupby('marketSegment')['num_mrt_stations_500m'].apply(lambda x : x.fillna(x.mode().iloc[0]))

In [64]:
len(dfnullmerge_geo)

657

In [65]:
dfmtest  = pd.concat([df_test, dfnullmerge_geo], ignore_index=True)

In [66]:
dfmtest.isna().sum()

property_key             0
contractDate             0
floorRange               0
typeOfArea               0
area                     0
propertyType             0
district                 0
marketSegment            0
num_schools_1km          0
num_supermarkets_500m    0
num_mrt_stations_500m    0
tenure                   0
dtype: int64

In [67]:
def contract_year_month(df):
    
    data = df['contractDate'].values
    years, months , datetime= [], [], []
    for i in data:
        year, month, _ = i.split('-')
        years.append(int(year))
        datetime.append(pd.to_datetime(i))
        months.append(int(month))
    return [years, months, datetime]

In [68]:
years, months,datetime = contract_year_month(dfmtest)
dfmtest['year'] = years
dfmtest['month'] = months
dfmtest['contractDate'] = datetime

In [69]:
cut_lables = ['small', 'medium', 'medium_large', 'large',]
cut_bins = [0, 200 , 400, 600, 1000]
dfmtest['area binning'] = pd.cut(dfmtest['area'], bins=cut_bins , labels=cut_lables)

In [70]:
df_onehot = pd.get_dummies(dfmtest, columns=['marketSegment', 'typeOfArea', 'area binning'])

In [71]:
len(train_x.columns)

19

In [72]:
df_onehot['propertyType'] = lablePropertyType.transform(df_onehot['propertyType'])
df_onehot['floorRange'] = lableFloorRange.transform(df_onehot['floorRange'])
df_onehot['tenure'] = lableTenure.transform(df_onehot['tenure'])

In [73]:
df_onehot.head(1)

,property_key,contractDate,floorRange,area,propertyType,district,num_schools_1km,num_supermarkets_500m,num_mrt_stations_500m,tenure,...,month,marketSegment_CCR,marketSegment_OCR,marketSegment_RCR,typeOfArea_Land,typeOfArea_Strata,area binning_small,area binning_medium,area binning_medium_large,area binning_large
0,p-38fe6afe9,2023-01-01,2,99.0,1,21.0,2.0,0.0,1.0,43,...,1,0,0,1,0,1,1,0,0,0


In [74]:
df_onehot = df_onehot.sort_values(by='contractDate', ignore_index=True, ascending=True)

In [86]:
df_onehot.to_csv('dftt.csv')

In [75]:
X_test = df_onehot[train_x.columns]

In [76]:
X_test

,year,month,floorRange,area,propertyType,district,num_schools_1km,num_supermarkets_500m,num_mrt_stations_500m,tenure,marketSegment_CCR,marketSegment_OCR,marketSegment_RCR,typeOfArea_Land,typeOfArea_Strata,area binning_small,area binning_medium,area binning_medium_large,area binning_large
0,2023,1,2,99.0,1,21.0,2.0,0.0,1.0,43,0,0,1,0,1,1,0,0,0
1,2023,1,4,88.0,1,3.0,2.0,1.0,1.0,60,0,0,1,0,1,1,0,0,0
2,2023,1,2,95.0,1,23.0,4.0,0.0,2.0,47,0,1,0,0,1,1,0,0,0
3,2023,1,1,82.0,1,17.0,5.0,0.0,0.0,121,0,1,0,0,1,1,0,0,0
4,2023,1,4,95.0,3,22.0,2.0,2.0,0.0,65,0,1,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2326,2023,3,3,142.0,3,27.0,3.0,2.0,1.0,65,0,1,0,0,1,1,0,0,0
2327,2023,3,1,73.0,1,18.0,0.0,0.0,0.0,62,0,1,0,0,1,1,0,0,0
2328,2023,3,1,73.0,1,14.0,6.0,1.0,2.0,61,0,0,1,0,1,1,0,0,0
2329,2023,3,1,73.0,0,10.0,6.0,4.0,4.0,65,1,0,0,0,1,1,0,0,0


In [77]:
price = rf.predict(X_test)

In [78]:
final_result = test.copy()
final_result['prediction'] = price.astype('int64')

In [79]:
final_result.to_csv('submission.csv', index=False)

In [80]:
final_result.isna().sum()

property_key    0
contractDate    0
prediction      0
dtype: int64

In [81]:
pd.read_csv('submission.csv')

,property_key,contractDate,prediction
0,p-ff93e87ab,2023-01-01,1261364
1,p-38fe6afe9,2023-01-01,1592350
2,p-fc9650179,2023-01-01,851369
3,p-59a09ad08,2023-01-01,1133391
4,p-808332e5c,2023-01-01,1138229
...,...,...,...
2326,p-56681c564,2023-03-01,1754083
2327,p-89a6ff5de,2023-03-01,906272
2328,p-920d750e9,2023-03-01,1178338
2329,p-e7146b43c,2023-03-01,1381578


In [82]:
1694021

1694021